# Example MM Corpus Handling

Here's how to create a MatrixMarket corpus using our tool and how to read it later using *gensim*.

In [1]:
import logging
import pandas as pd
import pickle

from gensim import corpora

logging.basicConfig(level=logging.INFO)

In [2]:
def unpickle(pkl):
    """Unpickle a gensim-created pickled object"""
    with open(pkl, 'rb') as infil:
        return pickle.load(infil)

## Running the Tool

Let's use a pretty small corpus: documents from 1991.

The switch `-p` becomes crucial for larger corpora because it allows saving the corpus in parallel. Of course, the I/O isn't in parallel, but documents are queued up (read, decompressed, and preprocessed) in parallel to keep the I/O pipeline fed.

Assuming we're executing this command from *notebooks/arxiv/*, we could run the following:

In [3]:
!python3 ../../corpora/arxiv/build-corpus.py -p -o example ../../data/arxiv/text/91*

2019-02-14 15:27:43,573:INFO: Initializing dictionary
2019-02-14 15:27:43,573:INFO: Using experimental get_texts_in_parallel()
2019-02-14 15:27:43,635:INFO: Caching the number of documents
2019-02-14 15:27:43,645:INFO: adding document #0 to Dictionary(0 unique tokens: [])
2019-02-14 15:27:44,214:INFO: built Dictionary(7436 unique tokens: ['a.a', 'a.r', 'abstract', 'addison', 'alter']...) from 111 documents (total 242570 corpus positions)
2019-02-14 15:27:44,217:INFO: saving Dictionary object under example.mm.dictionary.cpickle, separately None
2019-02-14 15:27:44,219:INFO: saved example.mm.dictionary.cpickle
2019-02-14 15:27:44,219:INFO: storing corpus in Matrix Market format to example.mm
2019-02-14 15:27:44,219:INFO: saving sparse matrix to example.mm
2019-02-14 15:27:44,219:INFO: Using experimental get_texts_in_parallel()
2019-02-14 15:27:44,295:INFO: PROGRESS: saving document #0
2019-02-14 15:27:44,829:INFO: saved 111x7436 matrix, density=7.491% (61832/825396)
2019-02-14 15:27:44,8

The tool creates a few files:

In [4]:
!ls -lh example*

-rw-rw-r-- 1 ec2-user ec2-user 581K Feb 14 15:27 example.mm
-rw-rw-r-- 1 ec2-user ec2-user 178K Feb 14 15:27 example.mm.dictionary.cpickle
-rw-rw-r-- 1 ec2-user ec2-user  526 Feb 14 15:27 example.mm.index
-rw-rw-r-- 1 ec2-user ec2-user 2.8K Feb 14 15:27 example.mm.metadata.cpickle


 - *example.mm* is the MatrixMarket corpus object
 - *example.mm.index* is the Matrix Market index file
 - *example.mm.dictionary.cpickle* is the serialized token-to-ID dictionary
 - *example.mm.metadata.cpickle* is the serialized dictionary of document IDs

The metadata is crucial, especially when using the `-p` switch, for identifying documents. When we read and save in parallel mode, document order might become nondeterministic.

## Loading the Corpus

The dictionary, metadata, and corpus can be accessed as follows:

In [5]:
dictionary = unpickle('./example.mm.dictionary.cpickle')
metadata = unpickle('./example.mm.metadata.cpickle')
corpus = corpora.MmCorpus('./example.mm')

INFO:gensim.corpora.indexedcorpus:loaded corpus index from ./example.mm.index
INFO:gensim.corpora._mmreader:initializing cython corpus reader from ./example.mm
INFO:gensim.corpora._mmreader:accepted corpus with 111 documents, 7436 features, 61832 non-zero entries


The *dictionary* and *metadata* objects are accessible using (0-indexed) token and document IDs:

In [6]:
print(f'Dictionary entry 0: {dictionary[0]}')
print(f'Metadata entry 0: {metadata[0]}')

Dictionary entry 0: a.a
Metadata entry 0: ('hep-th9108012',)


As usual with *gensim*, the corpus is an iterable:

In [7]:
for i, doc in enumerate(corpus[:5]):
    print(f'Document {i+1}: {len(doc)} unique tokens and {sum(int(x[1]) for x in doc)} words')

Document 1: 222 unique tokens and 413 words
Document 2: 258 unique tokens and 725 words
Document 3: 361 unique tokens and 758 words
Document 4: 317 unique tokens and 764 words
Document 5: 341 unique tokens and 902 words


Finally, we can combine all three corpus objects to produce (say) a *pandas* data frame:

In [8]:
tokens = pd.DataFrame([
    {
        'token': dictionary[token[0]],
        'document': metadata[i][0],
        'count': int(token[1])
    }
    for i, doc in enumerate(corpus[:100])
    for token in doc
])

In [9]:
counts = (
    tokens.groupby('token')['count']
        .agg('sum')
        .to_frame()
        .reset_index()
        .sort_values('count', ascending=False)
)

In [10]:
counts.head(n=20)

token  count
6097    theori   2395
2023     field   1983
3837     model   1754
2185  function   1706
2282     gener   1550
178    algebra   1457
4551       phy   1409
2262      gaug   1186
797       case   1185
5756    string   1156
100     action   1131
6060      term   1127
5695     state   1127
4303      oper   1097
4619     point   1068
5623     space   1040
2104      form    957
2082    follow    932
3703    matrix    855
4249    obtain    829